<a href="https://colab.research.google.com/github/namsalmaongoroeva/test1/blob/main/03_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задачи к Лекции 3

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [25]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.model_selection import StratifiedKFold # только для разбиения, можно заменить
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

df = pd.read_csv("/content/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")


# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,gender_bin
id,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,0


## Классы в Python

Нередко, возникает необходимость создания объектов с каким-нибудь внутренним поведением и состоянием. Примерами таких объектов являются классификаторы sklearn, массивы numpy и много другое. Такой объект можно объявить с помощью ключевого слова **class**

```python
class SomeObject:
    def __init__(self, depth):
        self.a = depth
        self.target = None
        
    def fit(self, data, target):
        self.target = data
        # magic
        return
    
    def predict(self, data):
        return self.target    
```

После этого в коде можно будет создать экземпляр данного класса
```python
a = SomeObject(depth=5)
a.fit(data, target)
a.predict(data)
```

## Задачи

**1. В sklearn на данный момент отсутствует функционал для построения деревьев решений из категориальных данных, поэтому его нужно сделать самостоятельно и проверить его работу. Что нужно сделать:**

* __создать классификатор используя только pandas, numpy и scipy. Необходимо его сделать самому, используя исключительно только numpy, pandas и scipy (запрещено использовать sklearn и прочие библиотеки). Напоминаю, что для категориальных данных операция < или > не имеют смысла (использовать только != и ==). Гиперпараметром данного классификатора должна быть максимальная глубина дерева.__
* __Проверить работу данного классификатора на наборе ("gender", "cholesterol", "gluc").__
* __С помощью кросс-валидации найти оптимальную глубину этого дерева. Для вашего классификатора GridSearchCV не подойдет, придется это сделать также самостоятельно.__
* __Нарисовать полученное дерево (я должен понять, как и откуда вы его нарисовали).__

Алгоритм работы классификатора:
 1. Перебираем все возможные признаки и смотрим либо неопределенность Джини, либо прирост информации. Это даст критерий разбиения в виде "признак == значение"
 2. Если выборка полученная при разбиении состоит из объектов одного класса (соответсвует нулевой энтропии), то данный лист просто возвращает значение этого класса.
 3. В противном случае, образуется новый узел и для него начинаем с пункта 1.
 4. Если достигли максимальной глубины, то вместа узла создаем лист, который возвращает самое вероятное значение.

__Замечание:__ в этой задаче не нужно использовать onehot-кодирование.

In [26]:
X = df[["gender_bin", "cholesterol", "gluc"]]
y = df["cardio"].values

print(X.head())
print(y[:5])

    gender_bin  cholesterol  gluc
id                               
0            1            1     1
1            0            3     1
2            0            3     1
3            1            1     1
4            0            1     1
[0 1 1 1 0]


In [27]:
# A lot of code here
# Класс узла дерева
class TreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, *, label=None):
        self.feature = feature      # Признак для разбиения
        self.value = value          # Значение признака
        self.left = left            # Левая ветвь (== значению)
        self.right = right          # Правая ветвь (!= значению)
        self.label = label          # Метка для листа

In [28]:
# Критерий Джини
def gini(y):
    counts = np.bincount(y)
    probs = counts / len(y)
    return 1 - np.sum(probs ** 2)

In [29]:
# Построение дерева
class CategoricalDecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        if len(np.unique(y)) == 1:
            return TreeNode(label=y[0])

        # Проверка на максимальную глубину
        if depth >= self.max_depth or X.shape[1] == 0:
            # Возвращаем наиболее вероятный класс
            label = Counter(y).most_common(1)[0][0]
            return TreeNode(label=label)

        best_feature, best_value, best_gini, splits = self._best_split(X, y)
        if best_feature is None:
            label = Counter(y).most_common(1)[0][0]
            return TreeNode(label=label)

        left_X, left_y, right_X, right_y = splits

        left_node = self._build_tree(left_X, left_y, depth + 1)
        right_node = self._build_tree(right_X, right_y, depth + 1)

        return TreeNode(best_feature, best_value, left=left_node, right=right_node)

    def _best_split(self, X, y):
        best_gini = 1.0
        best_feature = None
        best_value = None
        best_splits = None

        for feature in X.columns:
            values = X[feature].unique()
            for val in values:
                mask = (X[feature] == val)
                left_y = y[mask]
                right_y = y[~mask]
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                g = (len(left_y) * gini(left_y) + len(right_y) * gini(right_y)) / len(y)
                if g < best_gini:
                    best_gini = g
                    best_feature = feature
                    best_value = val
                    left_X = X[mask]
                    right_X = X[~mask]
                    best_splits = (left_X, left_y, right_X, right_y)

        return best_feature, best_value, best_gini, best_splits

    def predict_one(self, x):
        node = self.root
        while node.label is None:
            if x[node.feature] == node.value:
                node = node.left
            else:
                node = node.right
        return node.label

    def predict(self, X):
        return np.array([self.predict_one(row) for _, row in X.iterrows()])

In [30]:
# Кросс-валидация по оптимальной глубине
def cross_val_score_tree(X, y, max_depth, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []
    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        tree = CategoricalDecisionTree(max_depth=max_depth)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        acc = np.mean(y_pred == y_test)
        scores.append(acc)
    return np.mean(scores)

# Перебор глубин
results = {}
for depth in range(1, 7):
    score = cross_val_score_tree(X, y, max_depth=depth)
    print(f"Depth {depth}: Accuracy = {score:.4f}")
    results[depth] = score

best_depth = max(results, key=results.get)
print(f"Best depth: {best_depth}, Best score: {results[best_depth]:.4f}")

Depth 1: Accuracy = 0.5893
Depth 2: Accuracy = 0.5918
Depth 3: Accuracy = 0.5918
Depth 4: Accuracy = 0.5918
Depth 5: Accuracy = 0.5918
Depth 6: Accuracy = 0.5918
Best depth: 2, Best score: 0.5918


In [31]:
# Вывод дерева в текстовом виде
def print_tree(node, depth=0):
    indent = "  " * depth
    if node.label is not None:
        print(f"{indent}Leaf: class={node.label}")
    else:
        print(f"{indent}if {node.feature} == {node.value}:")
        print_tree(node.left, depth + 1)
        print(f"{indent}else:")
        print_tree(node.right, depth + 1)

# Пример использования:
tree = CategoricalDecisionTree(max_depth=best_depth)
tree.fit(X, y)
print_tree(tree.root)

if cholesterol == 1:
  if gluc == 2:
    Leaf: class=1
  else:
    Leaf: class=0
else:
  if cholesterol == 3:
    Leaf: class=1
  else:
    Leaf: class=1


**Комментарии:**



